In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
! git pull

## Add import path

In [ ]:
import os
import sys
import gc

In [ ]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
del module_path

## Organize imports

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from collections import OrderedDict

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from tqdm import tqdm

In [ ]:
import plotly.express as px

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.discriminant_analysis import (
    LinearDiscriminantAnalysis, 
    QuadraticDiscriminantAnalysis
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.datasets import (
    load_iris,
    load_wine
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    StandardScaler
)
from sklearn.metrics import (
    classification_report,
    confusion_matrix
)

In [ ]:
from scipy import stats
from scipy.interpolate import interp1d

In [ ]:
import torch

In [ ]:
from torch import nn
from torchvision import transforms
import torchvision.transforms.functional as F
from torchvision.utils import make_grid
from torchvision.datasets import MNIST

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

## Initialize Path

In [ ]:
PATH = Path('data')
model_dir = PATH / 'models'
model_path = model_dir / 'model_chechpoint.ckpt'
images_path = PATH / 'images'
images_path.mkdir(exist_ok=True, parents=True)
pumpkin_path = PATH / 'Pumpkin_Seeds_Dataset.xlsx'

## Model wrapper

In [ ]:
class NetWrapper(object):

    def __init__(self, net, transform):
        self.net = net.eval()
        self.transform = transform

    @torch.inference_mode()
    def forward(self, x, k=6):
        t = self.transform(x)
        r = self.net[: k](t)

        return r        
    
    def __call__(self, *args, **kwargs):
        return self.forward(*args, **kwargs)
    

## Load the model

In [ ]:
model = torch.load(model_path, map_location='cpu')

In [ ]:
model

In [ ]:
def clear_state_dict(state_dict):
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.1.', 'fc1.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.4.', 'fc2.')] = state_dict.pop(key)
    for key in list(state_dict.keys()):
        state_dict[key.replace('model.7.', 'fc3.')] = state_dict.pop(key)

    return state_dict

In [ ]:
state_dict = clear_state_dict(model['state_dict'])

In [ ]:
state_dict

In [ ]:
channels = 1
width = 28
height = 28
hidden_size = 16
num_classes = 10
in_features = channels * width * height

In [ ]:
net = nn.Sequential(OrderedDict([
    ('flatten', nn.Flatten()),
    ('fc1', nn.Linear(channels * width * height, hidden_size)),
    ('act1', nn.ReLU()),
    ('fc2', nn.Linear(hidden_size, hidden_size)),
    ('act2', nn.ReLU()),
    ('fc3', nn.Linear(hidden_size, num_classes)),
]))

In [ ]:
net

In [ ]:
net.load_state_dict(state_dict)

In [ ]:
net = net.eval()
net

## Helper functions

In [ ]:
def layer_V(data, net, k=5):
    V = list()
    X = list()
    with tqdm(data) as ds:
        for i, (x, y) in enumerate(ds):
            v = net(x, k=k).detach().numpy()[0]
            V.append(v)
            X.append(x)

    return np.array(V), X

In [ ]:
def loop_maxes(V, func, *args, **kwargs):
    with tqdm(V) as mstml:
        for i, v in enumerate(mstml):
            func(i, v, *args, **kwargs)

In [ ]:
def select_top(V, idx, thresh):
    tops = list()
    def add_to_top(i, v):
        if thresh <= v[idx]:
            tops.append(i)
    loop_maxes(V, lambda i, v: add_to_top(i, v))
    
    return tops

In [ ]:
def find_v_x(V, mrng, idx):
    mid = np.argmin(np.array(V)[mrng], axis=0)[idx]
    x_id = mrng[mid]
    v_x = V[x_id]

    return v_x, x_id

In [ ]:
def find_v_A(V, mrng):
    return np.minimum.reduce(np.array(V)[mrng])

In [ ]:
def find_G_x(V, v_x):
    with tqdm(V) as mstm:
        G_x = np.array([i for i, v in enumerate(mstm) if np.all(v_x <= v)])

    return G_x

In [ ]:
def show_img(ds, idx):
    plt.imshow(ds[idx][0])

In [ ]:
def show(
        imgs, 
        h=12, 
        w=12, 
        img_path=None
):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fig, axs = plt.subplots(
        ncols=len(imgs),
        figsize=(w, h),
        squeeze=False
    )
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    if img_path:
        plt.savefig(img_path)
    plt.show()

In [ ]:
def show_grid(
        G_A, 
        data, 
        nrow=8, 
        h=12, 
        w=12, 
        img_path=None
):
    to_tensor = transforms.ToTensor()
    A_gr = [to_tensor(data[i][0]) for i in G_A]
    grid = make_grid(A_gr, nrow=nrow)
    show(grid, h=h, w=w, img_path=img_path)

## Initialize MNIST dataset

In [ ]:
transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,)),
            ]
)

In [ ]:
data = MNIST(images_path, train=False, download=True)

In [ ]:
wnet = NetWrapper(net, transform)

In [ ]:
V_X, X_V = layer_V(data, wnet, k=5)

In [ ]:
np.argmax(V_X, axis=0)

In [ ]:
np.max(V_X, axis=0)

In [ ]:
V_X[3290, :]

In [ ]:
show_img(data, 8140)

## Alanyze maximum stimulus

In [ ]:
# th_A = 16
# th_B = 10

th_A = 8
th_B = 8

# th_A = 6
# th_B = 6

In [ ]:
G_A_v_A = select_top(V_X, 2, th_A)

In [ ]:
show_grid(G_A_v_A, data, nrow=48, h=16, w=32, img_path=images_path / f'G_A_v_A.png')

In [ ]:
v_a, a_id = find_v_x(V_X, G_A_v_A, 2)
v_a, a_id

In [ ]:
show_img(data, a_id)

In [ ]:
v_A = find_v_A(V_X, G_A_v_A)
v_A

In [ ]:
G_B_v_B = select_top(V_X, 5, thresh=th_B)

In [ ]:
show_grid(G_B_v_B, data, nrow=48, h=16, w=32, img_path=images_path / f'G_B_v_B.png')

In [ ]:
v_b, b_id = find_v_x(V_X, G_B_v_B, 5)
v_b, b_id

In [ ]:
show_img(data, b_id)

In [ ]:
v_B = find_v_A(V_X, G_B_v_B)
v_B

In [ ]:
G_A = find_G_x(V_X, v_A)
G_A

In [ ]:
show_grid(G_A, data, nrow=48, h=16, w=32, img_path=images_path / f'G_A.png')

In [ ]:
G_B = find_G_x(V_X, v_B)
G_B

In [ ]:
show_grid(G_B, data, nrow=48, h=16, w=32, img_path=images_path / f'G_B.png')

In [ ]:
D = np.intersect1d(G_A, G_B)
D

In [ ]:
show_grid(D, data, nrow=48, h=16, w=32, img_path=images_path / f'D.png')

In [ ]:
plt.imshow(data[6794][0])

In [ ]:
v_D = np.maximum(v_A, v_B)
v_D

In [ ]:
G_5_D = find_G_x(V_X, v_D)
G_5_D

In [ ]:
show_grid(G_5_D, data, nrow=48, h=16, w=32, img_path=images_path / f'G_5_D.png')

In [ ]:
U_X, X_U = layer_V(data, wnet, k=6)

In [ ]:
u_D = find_v_A(U_X, D)
u_D

In [ ]:
U_X[D]

In [ ]:
G_u_D = find_G_x(U_X, u_D)
G_u_D

In [ ]:
show_grid(G_u_D, data, nrow=48, h=16, w=32, img_path=images_path / 'G_u_D.png')

In [ ]:
plt.imshow(data[9390][0])

In [ ]:
G_u_D.shape

In [ ]:
y = list()
with tqdm(data) as dt:
    for _, y_l in dt:
        y.append(y_l)
y = np.array(y)

In [ ]:
y_3_idx = np.where(y == 3)

In [ ]:
len(y_3_idx)

In [ ]:
y_3_idx[0].shape, G_u_D.shape

In [ ]:
result = y_3_idx[0][~np.isin(y_3_idx[0], G_u_D)]
result.shape

In [ ]:
show_grid(result, data, nrow=48, h=16, w=32)

In [ ]:
np.count_nonzero(y[G_u_D] == 3), G_u_D.shape